<h1> Optimization electricity consumption </h1>

<h3> Imports </h3>

In [1]:
import math
import random
import copy
import numpy as np
import joblib
import time
import multiprocessing
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

%run preprocess_functions.py
%run Individual.py

<h3> Helper functions </h3>

In [2]:
def calc_consumption_from_planing(plan,mat,day_consumption):
    for house in plan:
        for machine in house.keys():
            for time_slot in house[machine]:
                day_consumption[time_slot] += mat[machine]["power"]
    return day_consumption

<h3> Recuit functions </h3>

In [3]:
#Not Perfect
def initial_solution(empty_plannings,mat):

    for house in range(len(empty_plannings)):
        for machine in empty_plannings[house].keys():
            if type(mat[machine]["nb_time_slot"]) == list:
                nb_time_slots = random.sample(mat[machine]["nb_time_slot"],1)[0]
            else:
                nb_time_slots = mat[machine]["nb_time_slot"]
            
            if machine in ['LL','SL','LV']:
                start_index = random.randint(16,47)
                for i in range(nb_time_slots):
                    empty_plannings[house][machine].append((start_index+i)%48)
            elif machine in ['TV','PL','FO']:
                hours_dict = {"TV":[38,47],"PL":[36,40],"FO":[36,40]}
                start_h,end_h = hours_dict[machine]
                empty_plannings[house][machine] = random.sample(range(start_h, end_h), nb_time_slots)
            else:
                if mat[machine]["is_sequencable"]:
                    for i in range(nb_time_slots):
                        index = random.randint(0,15)
                        while index in empty_plannings[house][machine]:
                            index = random.randint(0,15)
                else:
                    start_index = random.randint(0,15-nb_time_slots)
                    for i in range(nb_time_slots):
                        empty_plannings[house][machine].append(start_index+i)
                

    day_consumption = []
    for i in range(48):
        day_consumption.append(int(0))

    day_consumption = calc_consumption_from_planing(empty_plannings,mat,day_consumption)

    individual = Individual({"start":0,"end":15},empty_plannings,day_consumption)
    

    return individual

In [4]:
def get_neighbour_solution(solution: Individual):
    solution_neigh = copy.deepcopy(solution)
    solution_neigh.mutate()

    return solution_neigh

def cost_function(solution : Individual):
    return solution.getMax()

def metropolis(neighbour_solution, current_solution,temperature):
    return math.exp( -( abs( cost_function(current_solution) - cost_function(neighbour_solution) ) / temperature ) )

def update_temperature(temperature):
    return temperature * 0.99

In [5]:
#TMP
def plot_planning(planning,title):
    m_count = 0
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    for machine in planning.keys():
        m_count += 1
        x = []
        y = []
        for time in planning[machine]:
            x.append(time)
            y.append(machine)
        
        ax.scatter(x,y)
    display(fig)
    clear_output(wait=True)
    plt.pause(0.1)

In [6]:
def recuit_simule(temperature,current_solution):
    best_current_solution = copy.deepcopy(current_solution)
    print_count = 0
    count_iteration_without_improvment = 0

    start_t = time.time()
    while count_iteration_without_improvment < 1000:
        # print_count += 1
        # if print_count%100 == 0:
        #     print("\r",end="")
        #     print("**Recuit",print_count,count_iteration_without_improvment,"| Time :",time.time()-start_t,end="                ")
        #     # plot_planning(best_current_solution.plannings[0],"Testos")
        #     start_t = time.time()

        neighbour_solution = get_neighbour_solution(current_solution)

        if cost_function(neighbour_solution) < cost_function(current_solution) or random.random() < metropolis( neighbour_solution , current_solution , temperature):
            current_solution = neighbour_solution

        if cost_function(current_solution) < cost_function(best_current_solution):
            best_current_solution = copy.deepcopy(current_solution)
            count_iteration_without_improvment = 0
        else:
            count_iteration_without_improvment += 1

        temperature = update_temperature(temperature)

    
    return best_current_solution

<h3> Variables </h3>

In [7]:
matrice_type_puissance_par_demie_heure = {'LV':{"power":float(0.65),"nb_time_slot":2,"is_sequencable":False}, 
                                            'LL':{"power":float(1),"nb_time_slot":2,"is_sequencable":False}, 
                                            'SL':{"power":float(0.125),"nb_time_slot":8,"is_sequencable":False}, 
                                            'TV':{"power":float(0.05),"nb_time_slot":[1,2,3,4,5],"is_sequencable":False}, 
                                            'FG_1':{"power":float(0.1),"nb_time_slot":4,"is_sequencable":False}, 
                                            'CE_1':{"power":float(0.18),"nb_time_slot":12,"is_sequencable":False}, 
                                            'CG':{"power":float(0.1),"nb_time_slot":4,"is_sequencable":False}, 
                                            'FO':{"power":float(0.8),"nb_time_slot":[1,2],"is_sequencable":False}, 
                                            'PL':{"power":float(0.6),"nb_time_slot":[1,2],"is_sequencable":False}, 
                                            'FG_2':{"power":float(0.3),"nb_time_slot":4,"is_sequencable":False}, 
                                            'CE_2':{"power":float(0.25),"nb_time_slot":12,"is_sequencable":False}
                                            }

<h3> Main </h3>

In [8]:
data = load_csv("data/data.csv")
batch_empty_plannings = make_batches("data/data.csv","data/data_network.csv")

In [9]:
# solutions = []
# nb_batches = len(batch_empty_plannings)
# count_batches = 0
# for key in list(batch_empty_plannings.keys())[:3]:
#     batch = batch_empty_plannings[key]

#     print("Init Solution")
#     current_solution = initial_solution(batch,matrice_type_puissance_par_demie_heure)
#     print("Score init :",current_solution.getMax(),"| Batch size :",len(batch)," | Batch number :",str(count_batches)+"/"+str(nb_batches))

#     best_found_solution = recuit_simule(1000, current_solution)
#     curr_best = cost_function(best_found_solution)

#     # print(best_found_solution.day_consumption)
#     print("Score optimized",cost_function(best_found_solution))
#     solutions.append(best_found_solution)
#     count_batches += 1




In [10]:
# joblib.dump(best_found_solution.plannings,"out/Plannings.joblib")
# joblib.dump(best_found_solution.day_consumption,"out/Day_Consumption.joblib")
# joblib.dump(best_found_solution,"out/Solution.joblib")
# joblib.dump(solutions,"out/Solutions.joblib")

In [11]:
def worker(procnum,batch,key,return_dict):
    print("Init Solution, process :", procnum)
    init_solution = initial_solution(batch,matrice_type_puissance_par_demie_heure)
    # print("Score init",cost_function(init_solution),", process :", procnum)
    score_init = cost_function(init_solution)

    best_found_solution = recuit_simule(1000, init_solution)
    curr_best = cost_function(best_found_solution)

    # print(best_found_solution.day_consumption)
    print("Score init/optimized",score_init,cost_function(best_found_solution),", process :", procnum)
    return_dict[key] = [best_found_solution,init_solution]
    print("Process Done :",procnum)


manager = multiprocessing.Manager()
return_dict = manager.dict()
jobs = []

nb_batches = len(batch_empty_plannings)
count_batches = 0
for key in list(batch_empty_plannings.keys())[:10]:
    batch = batch_empty_plannings[key]
    p = multiprocessing.Process(target=worker, args=(count_batches,batch,key,return_dict))
    jobs.append(p)
    p.start()
    count_batches += 1
    

for proc in jobs:
    proc.join()

solutions = return_dict.values()
print(len(solutions))

Init Solution, process : Init Solution, process :0 1

Init Solution, process : Init Solution, process :2Init Solution, process :Init Solution, process :Init Solution, process : 
  534 Init Solution, process :


 6
7Init Solution, process :
Init Solution, process :  98

Score init/optimized 127.74999999999957 120.475 , process : 6
Process Done : 6
Score init/optimized 122.94999999999959 114.55 , process : 4
Process Done : 4
Score init/optimized 133.74999999999972 118.45 , process : 3
Process Done : 3
Score init/optimized 126.74999999999963 112.575 , process : 9
Process Done : 9
Score init/optimized 127.82499999999963 108.1 , process : 2
Process Done : 2
Score init/optimized 138.72499999999977 110.725 , process : 5
Process Done : 5
Score init/optimized 127.62499999999959 106.925 , process : 7
Process Done : 7
Score init/optimized 132.67499999999964 108.7 , process : 1
Process Done : 1
Score init/optimized 139.2249999999997 112.5 , process : 0
Process Done : 0
Score init/optimized 134.599

In [12]:
joblib.dump(solutions,"out/Solution_f10.joblib")

['out/Solution_f10.joblib']